In [1]:
import pandas as pd

In [2]:
from sshtunnel import SSHTunnelForwarder

AWS_IP_ADDRESS = '54.189.184.164'
AWS_USERNAME = 'luke'
SSH_KEY_PATH = '/users/luketibbott/.ssh/id_rsa'

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 56831


In [3]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = 'luke'     ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'luke' ## CHANGE THIS TO YOUR POSTGRES PASSWORD
POSTGRES_DBNAME = 'tennis'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

In [4]:
cnx.table_names()

['aus_ladies_2013',
 'aus_men_2013',
 'us_men_2013',
 'us_ladies_2013',
 'french_men_2013',
 'french_ladies_2013',
 'wimbledon_men_2013',
 'wimbledon_ladies_2013']

**Question 1:** Using the same tennis data, find the number of matches played by each player in each tournament. (Remember that a player can be present as both player1 or player2).

In [25]:
pd.read_sql_query('''
                    WITH t1 AS (SELECT DISTINCT al.player1 ALp1, COUNT(al.player1) ALc1
                                FROM aus_ladies_2013 al
                                GROUP BY 1),
                         t2 AS (SELECT DISTINCT fl.player1 FLp1, COUNT(fl.player1) FLc1
                                FROM french_ladies_2013 fl
                                GROUP BY 1),
                         t3 AS (SELECT DISTINCT usl.player1 USLp1, COUNT(usl.player1) USLc1
                                FROM us_ladies_2013 usl
                                GROUP BY 1),
                         t4 AS (SELECT DISTINCT wl.player1 WLp1, COUNT(wl.player1) WLc1
                                FROM wimbledon_ladies_2013 wl
                                GROUP BY 1),
                                
                         t5 AS (SELECT DISTINCT al.player2 ALp2, COUNT(al.player2) ALc2
                                FROM aus_ladies_2013 al
                                GROUP BY 1),
                         t6 AS (SELECT DISTINCT fl.player2 FLp2, COUNT(fl.player2) FLc2
                                FROM french_ladies_2013 fl
                                GROUP BY 1),
                         t7 AS (SELECT DISTINCT usl.player2 USLp2, COUNT(usl.player2) USLc2
                                FROM us_ladies_2013 usl
                                GROUP BY 1),
                         t8 AS (SELECT DISTINCT wl.player2 WLp2, COUNT(wl.player2) WLc2
                                FROM wimbledon_ladies_2013 wl
                                GROUP BY 1), 
                                
                         women_p1 AS (SELECT *
                                FROM t1
                                FULL OUTER JOIN t2
                                ON t1.ALp1 = t2.FLp1
                                FULL OUTER JOIN t3
                                ON t1.ALp1 = t3.USLp1 AND t2.FLp1 = t3.USLp1
                                FULL OUTER JOIN t4
                                ON t1.ALp1 = t4.WLp1 AND t2.FLp1 = t4.WLp1 AND t3.USLp1 = t4.WLp1),
                                
                         women_p2 AS (SELECT *
                                 FROM t5
                                 FULL OUTER JOIN t6
                                 ON t5.ALp2 = t6.FLp2
                                 FULL OUTER JOIN t7
                                 ON t5.ALp2 = t7.USLp2 AND t6.FLp2 = t7.USLp2
                                 FULL OUTER JOIN t8
                                 ON t5.ALp2 = t8.WLp2 AND t6.FLp2 = t8.WLp2 AND t7.USLp2 = t8.WLp2
                         )
                    
                    SELECT *
                    FROM women_p2
                  ''', cnx)

,alp2,alc2,flp2,flc2,uslp2,uslc2,wlp2,wlc2
0,Ana Ivanovic,4.0,Ana Ivanovic,4.0,None,NaN,None,NaN
1,Ayumi Morita,2.0,None,NaN,None,NaN,None,NaN
2,Polona Hercog,1.0,None,NaN,None,NaN,None,NaN
3,Shuai Peng,1.0,Shuai Peng,1.0,None,NaN,None,NaN
4,Zarina Diyas,1.0,None,NaN,None,NaN,None,NaN
5,Yung-Jan Chan,1.0,None,NaN,None,NaN,None,NaN
6,Flavia Pennetta,2.0,None,NaN,None,NaN,None,NaN
7,Anna Tatishvili,1.0,Anna Tatishvili,1.0,None,NaN,None,NaN
8,Maria Sharapova,4.0,Maria Sharapova,7.0,None,NaN,None,NaN
9,Jie Zheng,1.0,Jie Zheng,1.0,None,NaN,None,NaN


In [ ]:
pd.read_sql_query('''
                    WITH p1s AS
                        (SELECT al.player1 ALp1, am.player1 AMp1, um.player1 UMp1, ul.player1 ULp1,
                         fm.player1 FMp1, fl.player1 FLp1, wm.player1 WMp1, wl.player1 WLp1 
                        FROM aus_ladies_2013 al, aus_men_2013 am, us_men_2013 um, us_ladies_2013 ul,
                             french_men_2013 fm, french_ladies_2013 fl, wimbledon_men_2013 wm, 
                             wimbledon_ladies_2013 wl)
                             
                    SELECT *
                    FROM p1s;
                  ''', cnx)

In [ ]:
pd.read_sql_query('''
                    SELECT player1
                    FROM aus_ladies_2013 al, aus_men_2013 am, us_men_2013 usm, us_ladies_2013 usl,
                         french_men_2013 fm, french_ladies_2013 fl, wimbledon_men_2013 wm,
                         wimbledon_ladies_2013 wl
                  ''', cnx)

In [7]:
pd.read_sql_query('''
                    SELECT player1
                    FROM aus_ladies_2013 al
                    OUTER JOIN aus_men_2013 am
                    ON am.player1 = al.player1
                    OUTER JOIN us_men_2013 usm
                    ON us_ladies_2013 usl,
                         french_men_2013 fm, french_ladies_2013 fl, wimbledon_men_2013 wm,
                         wimbledon_ladies_2013 wl
                  ''', cnx)

ProgrammingError: (psycopg2.ProgrammingError) column reference "player1" is ambiguous
LINE 2:                     SELECT player1
                                   ^
 [SQL: '\n                    SELECT player1\n                    FROM aus_ladies_2013 al, aus_men_2013 am, us_men_2013 usm, us_ladies_2013 usl,\n                         french_men_2013 fm, french_ladies_2013 fl, wimbledon_men_2013 wm,\n                         wimbledon_ladies_2013 wl\n                  '] (Background on this error at: http://sqlalche.me/e/f405)

In [5]:
pd.read_sql_query('''
                    WITH t1 AS (
                        SELECT player1, COUNT(al.player1) count1, 
                        FROM aus_ladies_2013 al, aus_men_2013 am, us_men_2013 usm, us_ladies_2013 usl,
                             french_men_2013 fm, french_ladies_2013 fl, wimbledon_men_2013 wm,
                             wimbledon_ladies_2013 wl
                        GROUP BY 1),
                        
                        t2 AS (
                            SELECT al.player2 al_player2, COUNT(al.player2) count2
                            FROM aus_ladies_2013 al 
                            GROUP BY 1), 
                        
                        aus_ladies AS (
                            SELECT t1.al_player1, (t1.count1 + t2.count2) AS appearances_ausladies
                            FROM t1
                            JOIN t2
                            ON t1.al_player1 = t2.al_player2
                            ORDER BY 1),
                            
                            
                        SELECT * 
                        FROM aus_ladies;
                  ''', cnx)

,al_player1,appearances_ausladies
0,Agnieszka Radwanska,6
1,Alison Riske,3
2,Ana Ivanovic,5
3,Barbora Zahlavova Strycova,2
4,Belinda Bencic,2
5,Camila Giorgi,2
6,Casey Dellacqua,4
7,Dominika Cibulkova,7
8,Ekaterina Makarova,4
9,Elina Svitolina,3


In [12]:
pd.read_sql_query('''
                    SELECT player2, COUNT(player2)
                    FROM aus_ladies_2013
                    GROUP BY 1
                  ''', cnx)

,player2,count
0,Petra Kvitova,1
1,Anna Schmiedlova,2
2,Yvonne Meusburger,2
3,Tsvetana Pironkova,1
4,Luksika Kumkhum,1
5,Venus Williams,1
6,Irina Falconi,2
7,Barbora Zahlavova Strycova,1
8,Sara Errani,1
9,Irina-Camelia Begu,1
